In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Set up

In [ ]:
X_train = pd.read_csv("/kaggle/input/cs480680-spring2021/train_x.csv")
y_train = pd.read_csv("/kaggle/input/cs480680-spring2021/train_y.csv")
X_test = pd.read_csv("/kaggle/input/cs480680-spring2021/test_x.csv")

In [ ]:
y_train = y_train["label"]
y_train

# Data Processing

## Process -6 (missing)

Since I will be using an XGBoost, which can handle missing values, I will simply replace -6 with NaNs.

In [ ]:
X_train = X_train.replace(-6, np.nan)
X_train = X_train.drop("index", axis = 1)
X_train

In [ ]:
X_test = X_test.replace(-6, np.nan)
X_test = X_test.drop("index", axis = 1)
X_test

## Process -9 (not applicable)

I tried adding additional columns that indicated when something was not applicable - this reduced my accuracy :c so I left it out. I left the code commented out below to show what I had attempted to do.

In [ ]:
#X_train.iloc[:, :12].describe()

In [ ]:
#X_train.iloc[:, 11:].describe()

Only CITSHP and MIL have -9 values

In [ ]:
# X_train["CITSHP_NA"] = np.where(X_train["CITSHP"] == -9, 1, 0)
# X_train["MIL_NA"] = np.where(X_train["MIL"] == -9, 1, 0)
# X_train

In [ ]:
# X_test["CITSHP_NA"] = np.where(X_test["CITSHP"] == -9, 1, 0)
# X_test["MIL_NA"] = np.where(X_test["MIL"] == -9, 1, 0)
# X_test

# Additional Variables

## Sum Income

In this section, I tried summing all income variables since I know the tree-based XGBoost is not very good at addition between variables - unfortunately this also decreased my accuracy so I had to leave it out as well :c. Again, my code remains below.

In [ ]:
# X_train["ALL_INCOME"] = X_train["WAGP"] + X_train["SEMP"] + X_train["SSP"] + X_train["SSIP"] 
# + X_train["PAP"] + X_train["RETP"] + X_train["INTP"] + X_train["OIP"]
# X_train

In [ ]:
# X_test["ALL_INCOME"] = X_test["WAGP"] + X_test["SEMP"] + X_test["SSP"] + X_test["SSIP"] 
# + X_test["PAP"] + X_test["RETP"] + X_test["INTP"] + X_test["OIP"]
# X_test

# XGBoost

In [ ]:
from xgboost import *
from sklearn.model_selection import *

In [ ]:
data_dmatrix = DMatrix(data=X_train,label=y_train)

## Tune model

In [ ]:
params = {"learning_rate":0.300000012, 'objective':'reg:logistic'}

cv_results = cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="error", as_pandas=True, seed=123)

In [ ]:
cv_results.apply(lambda x: 1-x).mean()

In [ ]:
# params = { 'max_depth': [3, 5, 6, 10, 15, 20],
#            'learning_rate': [0.01, 0.1, 0.2, 0.3],
#            'subsample': np.arange(0.5, 1.0, 0.1),
#            'colsample_bytree': np.arange(0.4, 1.0, 0.1),
#            'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
#            'n_estimators': [100, 500, 1000]}

# xgb = XGBClassifier(seed = 0)

# clf = RandomizedSearchCV(estimator=xgb,
#                          param_distributions=params,
#                          scoring='accuracy',
#                          n_iter=25,
#                          verbose=1)

# clf.fit(X_train, y_train)

# print("Best parameters:", clf.best_params_)
# print("Best Score: ", clf.best_score_)

Best parameters: { 'max_depth': 10,
            'learning_rate': 0.01,
            'subsample': 0.9,
            'colsample_bytree': 0.8,
            'colsample_bylevel': 0.7,
            'n_estimators': 900}

In [ ]:
# params = { 'max_depth': [9, 10, 11],
#            'learning_rate': [0.005, 0.01, 0.02],
#            'subsample': [0.9],
#            'colsample_bytree': [0.8],
#            'colsample_bylevel': [0.7],
#            'n_estimators': [900, 1000]}

# xgb = XGBClassifier(seed = 0)

# clf = GridSearchCV(estimator=xgb,
#                          param_grid=params,
#                          scoring='accuracy',
#                          verbose=1)

# clf.fit(X_train, y_train)

# print("Best parameters:", clf.best_params_)
# print("Best Score: ", clf.best_score_)

Best parameters: {'colsample_bylevel': 0.7, 'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 900, 'subsample': 0.9}

Best Score:  0.717751559533496

In [ ]:
# params = { 'max_depth': [8, 9],
#            'learning_rate': [0.01],
#            'subsample': [0.8, 0.9],
#            'colsample_bytree': [0.7, 0.8, 0.9],
#            'colsample_bylevel': [0.6, 0.7, 0.8],
#            'n_estimators': [900]}

# xgb = XGBClassifier(seed = 0)

# clf = GridSearchCV(estimator=xgb,
#                          param_grid=params,
#                          scoring='accuracy',
#                          verbose=1)

# clf.fit(X_train, y_train)

# print("Best parameters:", clf.best_params_)
# print("Best Score: ", clf.best_score_)

**Best parameters: {'colsample_bylevel': 0.7, 'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 900, 'subsample': 0.8}**

**Best Score:  0.7180499050718742**

Public score = 0.71803

I tried further fine-tuning, but they gave worse results on the public test data, so these are the best parameters I got. It is also very close to the public score, so I think this will be consistent for the private score.

## Fit Tuned Model

In [ ]:
xgb = XGBClassifier(max_depth = 9, learning_rate = 0.01, subsample = 0.8, colsample_bytree = 0.7,
                   colsample_bylevel = 0.7, n_estimators = 900)
xgb.fit(X_train, y_train)

# Submit Prediction

In [ ]:
predictions = xgb.predict(X_test)
predictions

In [ ]:
submission = pd.DataFrame({'index': X_test.index, 'label': predictions})
submission.to_csv('results.csv', index=False)